In [52]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import json

# Set up directories and paths
model_path = 'C:/Users/josej/Thesis/models_test/inceptionv3.h5'  # Path to the saved model
class_indices_path = 'C:/Users/josej/Thesis/class_indices.json'   # Path to the class indices JSON
test_folder = 'C:/Users/josej/Thesis/test_images'  # Folder containing test images

# Load the saved model
model = tf.keras.models.load_model(model_path)

# Load class indices from JSON file
with open(class_indices_path, 'r') as f:
    class_indices = json.load(f)

# Inverse the class indices to get label from index
indices_to_class = {v: k for k, v in class_indices.items()}

# Function to preprocess images and make predictions
def predict_image(image_path, model):
    img = image.load_img(image_path, target_size=(224, 224))  # Load image and resize to 224x224
    img_array = image.img_to_array(img) / 255.0  # Convert to array and normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Predict the class
    predictions = model.predict(img_array)
    predicted_index = np.argmax(predictions)
    predicted_label = indices_to_class[predicted_index]
    
    return predicted_label

# Loop through all images in the test folder
for img_file in os.listdir(test_folder):
    img_path = os.path.join(test_folder, img_file)
    
    # Skip non-image files
    if not img_file.lower().endswith(('png', 'jpg', 'jpeg')):
        continue
    
    # Predict the label for the image
    predicted_label = predict_image(img_path, model)
    
    print(f"Image: {img_file} - Predicted Label: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Image: beagle_0.jpeg - Predicted Label: English_foxhound
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Image: beagle_1.jpeg - Predicted Label: English_foxhound
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Image: beagle_2.jpeg - Predicted Label: beagle
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Image: beagle_4.jpeg - Predicted Label: beagle
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Image: n02109961_1076.jpg - Predicted Label: Siberian_husky
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Image: n02109961_11511.jpg - Predicted Label: Siberian_husky
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Image: n02109961_12107.jpg - Predicted Label: Eskimo_dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Image: n02109961_1235.jpg - Predicted Label: malamute
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Image: n02109961_13077.jpg - Predicted Label: Eskimo_dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Image: n02109961_16491.jpg - Predicted Label: Boston_bull
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Image: n02109961_16718.jpg -

In [3]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import json

# Set up directories and paths
model_path = 'C:/Users/josej/Thesis/models_test/inceptionv3.h5'  # Path to the saved model
class_indices_path = 'C:/Users/josej/Thesis/class_indices.json'   # Path to the class indices JSON
test_folder = 'C:/Users/josej/Thesis/test_images'  # Folder containing subfolders of test images

# Paths for original train/val folders and the target train/val folders for images_75
train_folder = 'C:/Users/josej/Thesis/images/train'
val_folder = 'C:/Users/josej/Thesis/images/val'
target_train_folder = 'C:/Users/josej/Thesis/images_75/train'
target_val_folder = 'C:/Users/josej/Thesis/images_75/val'

# Create target directories if they don't exist
os.makedirs(target_train_folder, exist_ok=True)
os.makedirs(target_val_folder, exist_ok=True)

# Load the saved model
model = tf.keras.models.load_model(model_path)

# Load class indices from JSON file
with open(class_indices_path, 'r') as f:
    class_indices = json.load(f)

# Inverse the class indices to get label from index
indices_to_class = {v: k for k, v in class_indices.items()}

# Function to preprocess images and make predictions
def predict_image(image_path, model):
    img = image.load_img(image_path, target_size=(224, 224))  # Load image and resize to 224x224
    img_array = image.img_to_array(img) / 255.0  # Convert to array and normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Predict the class without showing progress output
    predictions = model.predict(img_array, verbose=0)
    predicted_index = np.argmax(predictions)
    predicted_label = indices_to_class[predicted_index]
    
    return predicted_label

# Track folders with accuracy greater than 75%
folders_above_75 = []

# Loop through each subfolder in the test folder
for folder in os.listdir(test_folder):
    folder_path = os.path.join(test_folder, folder)
    
    if os.path.isdir(folder_path):
        correct_predictions = 0
        total_images = 0
        
        # Loop through each image in the subfolder
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            
            # Skip non-image files
            if not img_file.lower().endswith(('png', 'jpg', 'jpeg')):
                continue
            
            total_images += 1
            predicted_label = predict_image(img_path, model)
            
            # Compare the predicted label with the folder name (correct label)
            if predicted_label == folder:
                correct_predictions += 1
        
        # Calculate accuracy
        accuracy = (correct_predictions / total_images) * 100 if total_images > 0 else 0
        
        # Output the results for the folder
        print(f"Folder: {folder} - Total Images: {total_images} - Correct: {correct_predictions} - Accuracy: {accuracy:.2f}%")
        
        # Add folder to list if accuracy is greater than 75%
        if accuracy > 75:
            folders_above_75.append(folder)

# Function to copy folders with accuracy > 75%
def copy_folders(source_folder, target_folder, folder_names):
    for folder_name in folder_names:
        src_folder = os.path.join(source_folder, folder_name)
        dest_folder = os.path.join(target_folder, folder_name)
        
        if os.path.exists(src_folder):
            shutil.copytree(src_folder, dest_folder, dirs_exist_ok=True)
            print(f"Copied: {folder_name} from {source_folder} to {target_folder}")

# Copy folders from train and val where accuracy > 75%
copy_folders(train_folder, target_train_folder, folders_above_75)
copy_folders(val_folder, target_val_folder, folders_above_75)


Folder: affenpinscher - Total Images: 30 - Correct: 20 - Accuracy: 66.67%
Folder: Afghan_hound - Total Images: 48 - Correct: 43 - Accuracy: 89.58%
Folder: African_hunting_dog - Total Images: 34 - Correct: 33 - Accuracy: 97.06%
Folder: Airedale - Total Images: 41 - Correct: 40 - Accuracy: 97.56%
Folder: American_Staffordshire_terrier - Total Images: 33 - Correct: 25 - Accuracy: 75.76%
Folder: Appenzeller - Total Images: 31 - Correct: 15 - Accuracy: 48.39%
Folder: Australian_terrier - Total Images: 40 - Correct: 31 - Accuracy: 77.50%
Folder: basenji - Total Images: 42 - Correct: 34 - Accuracy: 80.95%
Folder: basset - Total Images: 35 - Correct: 35 - Accuracy: 100.00%
Folder: beagle - Total Images: 39 - Correct: 20 - Accuracy: 51.28%
Folder: Bedlington_terrier - Total Images: 37 - Correct: 36 - Accuracy: 97.30%
Folder: Bernese_mountain_dog - Total Images: 44 - Correct: 40 - Accuracy: 90.91%
Folder: Blenheim_spaniel - Total Images: 38 - Correct: 29 - Accuracy: 76.32%
Folder: bloodhound - T